In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import load_model
import time

# Data preprocessing

In [3]:
train_data = pd.read_csv("csv_files/train.csv")
# Obtain features and label -> X, Y
X, Y = train_data.drop(["moneyness", "price"], axis=1), train_data["price"]

# Normalize the data -> X_normalized, Y_normalized
## Normalize the model parameters in a domain
def custom_min_max_normalization(x, xmin, xmax):
    return (2 * x - (xmax + xmin)) / (xmax - xmin)

model_parameters = X.drop(["tau", "stockPrice", "strike"], axis=1)
option_properties = X.drop(["varsigma", "kappa", "delta", "v0", "rho"], axis=1)

min_vals = pd.Series({ 
    'varsigma': 0.01,
    'kappa': 0,
    'v0': 0.03,
    'delta': 0.01,
    'rho': -0.9
})

max_vals = pd.Series({
    'varsigma': 0.5,
    'kappa': 3.0,
    'v0': 0.15,
    'delta': 0.8,
    'rho': -0.2
})

normalized_model_parameters = pd.DataFrame()
for column in model_parameters.columns:
    normalized_model_parameters[column] = custom_min_max_normalization(
        model_parameters[column], 
        min_vals[column], 
        max_vals[column]
    )

scaler = MinMaxScaler() 
normalized_option_properties = scaler.fit_transform(option_properties)
normalized_option_properties = pd.DataFrame(normalized_option_properties, columns=option_properties.columns)

X_normalized = pd.concat([normalized_model_parameters, normalized_option_properties], axis=1).values

Y_normalized = scaler.fit_transform(Y.values.reshape(-1, 1))
Y_normalized = pd.Series(Y_normalized.flatten(), name=Y.name).values

In [4]:
# Define sequences of days
num_features = X.shape[1]
num_days = 53 # number of days
num_samples = len(X)
num_samples_per_day = num_samples // num_days

X_days = [X_normalized[i*num_samples_per_day : (i+1)*num_samples_per_day] for i in range(num_days)]
Y_days = [Y_normalized[i*num_samples_per_day : (i+1)*num_samples_per_day] for i in range(num_days)]

X_seq = np.stack(X_days, axis=1)
Y_seq = np.stack(Y_days, axis=1)

# Calibration Data

In [5]:
# Collect the data
calibrate = pd.read_csv("csv_files/syntheticCalibration.csv")
calibrate = round(calibrate,2)
calibrate

,varsigma,kappa,delta,v0,rho,tau,stockPrice,strike,moneyness,price
0,0.1,1,0.2,0.04,-0.75,1.0,189.00,151,1.25,48.93
1,0.1,1,0.2,0.04,-0.75,1.0,189.00,153,1.24,47.35
2,0.1,1,0.2,0.04,-0.75,1.0,189.00,155,1.22,45.79
3,0.1,1,0.2,0.04,-0.75,1.0,189.00,157,1.20,44.25
4,0.1,1,0.2,0.04,-0.75,1.0,189.00,159,1.19,42.73
...,...,...,...,...,...,...,...,...,...,...
29145,0.1,1,0.2,0.04,-0.75,1.0,191.61,241,0.80,5.65
29146,0.1,1,0.2,0.04,-0.75,1.0,191.61,243,0.79,5.26
29147,0.1,1,0.2,0.04,-0.75,1.0,191.61,245,0.78,4.90
29148,0.1,1,0.2,0.04,-0.75,1.0,191.61,247,0.78,4.55


In [6]:
# Supondo que train_data seja o seu DataFrame
calibrate.describe()

,varsigma,kappa,delta,v0,rho,tau,stockPrice,strike,moneyness,price
count,2.915000e+04,29150.0,2.915000e+04,29150.00,29150.00,29150.000000,29150.000000,29150.000000,29150.000000,29150.000000
mean,1.000000e-01,1.0,2.000000e-01,0.04,-0.75,1.000000,185.365283,200.000000,0.946906,18.844054
std,1.387803e-17,0.0,2.775605e-17,0.00,0.00,0.431683,5.632691,28.862234,0.143235,14.222232
min,1.000000e-01,1.0,2.000000e-01,0.04,-0.75,0.000000,172.300000,151.000000,0.690000,-0.000000
25%,1.000000e-01,1.0,2.000000e-01,0.04,-0.75,0.690000,180.940000,175.000000,0.820000,6.750000
50%,1.000000e-01,1.0,2.000000e-01,0.04,-0.75,1.000000,186.740000,200.000000,0.930000,15.915000
75%,1.000000e-01,1.0,2.000000e-01,0.04,-0.75,1.310000,189.430000,225.000000,1.060000,29.240000
max,1.000000e-01,1.0,2.000000e-01,0.04,-0.75,2.000000,196.620000,249.000000,1.300000,61.650000


In [7]:
CalX, CalY = calibrate.drop(["moneyness","price"], axis=1), calibrate["price"]

In [8]:
# Normalize CalX
Cal_model_parameters = CalX.drop(["tau", "stockPrice", "strike"], axis=1)
Cal_option_properties = CalX.drop(["varsigma", "kappa", "delta", "v0", "rho"], axis=1)

normalized_Cal_model_parameters = pd.DataFrame()
for column in Cal_model_parameters.columns:
    normalized_Cal_model_parameters[column] = custom_min_max_normalization(
        Cal_model_parameters[column], 
        min_vals[column], 
        max_vals[column]
    )

# Fit the scaler with option_properties from the training data
scaler_option_properties = MinMaxScaler() 
scaler_option_properties.fit(option_properties)

# Transform the option_properties from the Cal data
normalized_Cal_option_properties = scaler_option_properties.transform(Cal_option_properties)
normalized_Cal_option_properties = pd.DataFrame(normalized_Cal_option_properties, columns=Cal_option_properties.columns)

CalX_normalized = pd.concat([normalized_Cal_model_parameters, normalized_Cal_option_properties], axis=1).values

# Now the scaler for Y
scaler_Y = MinMaxScaler()
scaler_Y.fit(Y.values.reshape(-1, 1))

# Normalize CalY
CalY_normalized = scaler_Y.transform(CalY.values.reshape(-1, 1))  # Use transform, not fit_transform
CalY_normalized = pd.Series(CalY_normalized.flatten(), name=CalY.name).values

In [9]:
# Define sequences of days
Calnum_features = CalX.shape[1]
Calnum_days = 53 # number of days
Calnum_samples = len(CalX)
Calnum_samples_per_day = Calnum_samples // Calnum_days

CalX_days = [CalX_normalized[i*Calnum_samples_per_day  : (i+1)*Calnum_samples_per_day ] for i in range(Calnum_days)]
CalY_days = [CalY_normalized[i*Calnum_samples_per_day  : (i+1)*Calnum_samples_per_day ] for i in range(Calnum_days)]

CalX_seq = np.stack(CalX_days, axis=1)
CalY_seq = np.stack(CalY_days, axis=1)

# Checking the network

In [10]:
lstm = load_model('BestLSTM.keras')

In [11]:
print("CalEvaluate: ", lstm.evaluate(CalX_seq, CalY_seq))
Calpred = lstm.predict(CalX_seq).flatten(order='F').reshape(-1,1)
Calpred_denormalized = scaler_Y.inverse_transform(Calpred.flatten(order='F').reshape(-1,1))
CalY_denormalized = scaler_Y.inverse_transform(CalY_seq.flatten(order='F').reshape(-1,1))
mse = mean_squared_error(CalY_denormalized, Calpred_denormalized)
print("MSE desnormalized", mse)

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 1.8344e-06
CalEvaluate:  1.670205620030174e-06
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
MSE desnormalized 0.016919889146546095


# PSO

In [10]:
def PSO(N, D, g, pMarket, fixed_input, lstm, I, S, omega, c1, c2, stopping_criteria):
    start_time = time.time()  # Start the timer
    # Initialize particles and velocities
    Omega = np.zeros((N,D))
    v = np.zeros((N,D))
    for k in range(N):
        for l in range(D):
            Omega[k, l] = np.round(I[l] + np.random.rand()*(S[l]-I[l]), 2)
            v[k, l] = 0

    # Initialize personal best positions and global best position
    Pbest = Omega.copy()
    g_values = [g(pMarket,np.tile(x, (fixed_input.shape[0],1)),fixed_input,lstm) for x in Pbest]
    Gbest = Pbest[np.argmin(g_values)]

    t = 0
    
    while not stopping_criteria(Omega, t):
        print("iteration: ", t)  
        r1, r2 = np.random.rand(), np.random.rand()
        print(Gbest)
        print(Pbest)

        for k in range(N):
            for l in range(D):
                # Update velocity and position
                v[k, l] = np.round(omega * v[k, l] + c1 * r1 * (Pbest[k, l] - Omega[k, l]) + c2 * r2 * (Gbest[l] - Omega[k, l]), 2)
                Omega[k, l] += v[k, l]
                ## ensure Omega stays within [I, S]
                if Omega[k, l] < I[l]:
                    Omega[k, l] = np.round(S[l] - (I[l] - Omega[k, l]) % (S[l] - I[l]), 2)
                elif Omega[k, l] > S[l]:
                    Omega[k, l] = np.round(I[l] + (Omega[k, l] - S[l]) % (S[l] - I[l]), 2)

            
            # Update personal best position
            g_Omega = g(pMarket,np.tile(Omega[k], (fixed_input.shape[0], 1)),fixed_input,lstm)
            if g_Omega < g_values[k]:
                Pbest[k] = Omega[k]
                g_values[k] = g_Omega

            # Update global best position
            if g_values[k] < g(pMarket,np.tile(Gbest, (fixed_input.shape[0], 1)),fixed_input,lstm):
                Gbest = Pbest[k]      
        t += 1
        print(Omega)

    end_time = time.time()  # Stop the timer
    execution_time = np.round(end_time - start_time, 2)  # Calculate the execution time
    
    return Gbest,t,execution_time

def stopping_criteria(Omega, t, max_iter=20):
    unique_elements = {tuple(vec) for vec in Omega}
    if len(unique_elements) == 1 or t > max_iter:
        return True
    return False

# Objective Function

In [13]:
def obj_function(pMarket, Omega, fixed_input, lstm):
    ObjX = np.concatenate((Omega, fixed_input), axis=1)

    # Define sequences of days
    num_days = 53 # number of days
    num_samples = len(ObjX)
    num_samples_per_day = num_samples // num_days

    ObjX = pd.DataFrame(ObjX, columns = ['varsigma', 'kappa', 'delta', 'v0', 'rho', 'tau', 'stockPrice', 'strike'])
    Obj_model_parameters = ObjX.drop(["tau", "stockPrice", "strike"], axis=1)
    Obj_option_properties = ObjX.drop(["varsigma", "kappa", "delta", "v0", "rho"], axis=1)
    
    normalized_Obj_model_parameters = pd.DataFrame()
    for column in Obj_model_parameters.columns:
        normalized_Obj_model_parameters[column] = custom_min_max_normalization(
            Obj_model_parameters[column], 
            min_vals[column], 
            max_vals[column]
        )

    # Transform the option_properties from the Cal data
    normalized_Obj_option_properties = scaler_option_properties.transform(Obj_option_properties)
    normalized_Obj_option_properties = pd.DataFrame(normalized_Obj_option_properties, columns=Obj_option_properties.columns)

    ObjX_normalized = pd.concat([normalized_Obj_model_parameters, normalized_Obj_option_properties], axis=1).values

    ObjX_days = [ObjX_normalized[i*num_samples_per_day : (i+1)*num_samples_per_day] for i in range(num_days)]
    ObjX_seq = np.stack(ObjX_days, axis=1)

    Objpred = lstm.predict(ObjX_seq).flatten(order='F').reshape(-1,1)
    pNetwork = scaler_Y.inverse_transform(Objpred.flatten(order='F').reshape(-1,1))
    
    return mean_squared_error(pNetwork,pMarket)

# Calibration
Optimal: (varsigma, kappa,	delta,	v0,	rho) = (0.1,	1,	0.2,	0.04,	-0.75)

In [14]:
D = 5
fixed_input = CalX.values[:,5:]
Omega = CalX.values[:,0:5]

obj_function(CalY.values,Omega,fixed_input,lstm)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


0.0169198891465461

In [ ]:
# Run the PSO algorithm
Gbest, t, timer = PSO(N=100, D=5, g=obj_function, pMarket=CalY.values, fixed_input=fixed_input, lstm=lstm, I=[0.01, 0 , 0.01, 0.03, -0.9], S=[0.5, 3, 0.8, 0.15, -0.2], omega=0.8, c1=1.2, c2=1.5, stopping_criteria=stopping_criteria)
print("best position: ", Gbest, ", best cost: ", obj_function(CalY.values, np.tile(Gbest, (fixed_input.shape[0], 1)), fixed_input,lstm), "iterations: ", t, "time: ", timer)